In [13]:
import pyranges as pr
import seaborn as sns
import yaml
import os
import sys
from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))+'/scripts/'
print(p)
sys.path.append(p)

from sm_utils import *


/Users/fairliereese/Documents/programming/mele_lab/projects/240706_pantranscriptome_cerberus_gtf_merge/scripts/


In [14]:
config = load_config()
od = '../../../'

In [15]:
analysis = 'espresso_pseudomasked_genomic'
tool = 'espresso'
config_tsv = od+f'snakemake/config_{analysis}_expression.tsv'
meta_df = parse_config(config_tsv)
meta_df['analysis'] = analysis
tech_rep = ['GM10493_1']

## Assign novel genes

In [98]:
ifile = od+f'data/{analysis}/raw/2_PY2_GM10493.gtf'
bed = expand(od+config['fmt']['novel_gene_merge_bed'], 
       analysis=analysis)[0]


In [108]:
gtf_df = rename_novel_genes(ifile, bed, 'test', tool)

In [107]:
def rename_novel_genes(ifile, bed, ofile, tool):
    """
    Rename novel genes based on the name of overlapping bed regions
    in a GTF file
    """
    
    # get entries w/ novel GIDs
    gtf_df = pr.read_gtf(ifile, as_df=True)
    nov_gids = get_gtf_novel_genes(gtf_df, tool)
    nov_df = gtf_df.loc[gtf_df.gene_id.isin(nov_gids)]
    l1 = len(nov_df.index)

    # use tid or gid based on tool
    if tool == 'iq':
        id_col = 'gene_id'
    elif tool == 'espresso':
        id_col = 'transcript_id'

    # merge w/ bed
    df = pr.read_bed(bed)
    nov_df = pr.PyRanges(nov_df)
    nov_df = nov_df.join(df,
                          how='left',
                          strandedness='same',
                          preserve_order=True)

    # sanity checks
    nov_df = nov_df.df
    l2 = len(nov_df.index)
    assert l1 == l2
    assert len(nov_df.loc[nov_df.Name.isnull()].index) == 0


    # get a dict to map sample-level novel gene ids to cross-sample novel gene ids
    id_dict = dict([(o,n) for o,n in zip(nov_df[id_col].tolist(),
                                         nov_df['Name'].tolist())])
    nov_df[[id_col, 'Name']].drop_duplicates().head()

    # now update full gtf
    nov_ids = nov_df[id_col].unique().tolist()
    inds = gtf_df.loc[gtf_df[id_col].isin(nov_ids)].index
    gtf_df.loc[inds, 'gene_id'] = gtf_df.loc[inds, id_col].map(id_dict)
    
    return gtf_df

In [99]:
# get entries w/ novel GIDs
gtf_df = pr.read_gtf(ifile, as_df=True)
nov_gids = get_gtf_novel_genes(gtf_df, tool)
nov_df = gtf_df.loc[gtf_df.gene_id.isin(nov_gids)]
l1 = len(nov_df.index)

In [100]:
# use tid or gid based on tool
if tool == 'iq':
    id_col = 'gene_id'
elif tool == 'espresso':
    id_col = 'transcript_id'


In [101]:
# merge w/ bed
df = pr.read_bed(bed)
nov_df = pr.PyRanges(nov_df)
nov_df = nov_df.join(df,
                      how='left',
                      strandedness='same',
                      preserve_order=True)


In [102]:
nov_df = nov_df.df
nov_df.loc[nov_df.Name.isnull()]
nov_df.loc[nov_df.Name == 'NA']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,transcript_id,gene_id,exon_number,Start_b,End_b,Name,Score_b,Strand_b


In [103]:
# get a dict to map sample-level novel gene ids to cross-sample novel gene ids
id_dict = dict([(o,n) for o,n in zip(nov_df[id_col].tolist(),
                                     nov_df['Name'].tolist())])
nov_df[[id_col, 'Name']].drop_duplicates().head()

# now update full gtf
nov_ids = nov_df[id_col].unique().tolist()
inds = gtf_df.loc[gtf_df[id_col].isin(nov_ids)].index
gtf_df.loc[inds, 'gene_id'] = gtf_df.loc[inds, id_col].map(id_dict)

In [104]:
l  = gtf_df.loc[(gtf_df.gene_id.str.contains('novel'))].gene_id.unique().tolist()
print(len(l))

1360


In [109]:
gtf_df.loc[gtf_df[id_col] == 'ESPRESSO:GL000205.2:9:0']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,transcript_id,gene_id,exon_number
213606,GL000205.2,novel_isoform,transcript,137791,145075,.,-,.,ESPRESSO:GL000205.2:9:0,novel_gene_4,NaN
213607,GL000205.2,novel_isoform,exon,137791,138168,.,-,.,ESPRESSO:GL000205.2:9:0,novel_gene_4,1
213608,GL000205.2,novel_isoform,exon,144905,145075,.,-,.,ESPRESSO:GL000205.2:9:0,novel_gene_4,2


In [3]:
def make_hier_entry(df, how='t'):
    """
    kind {'g','t'}
    """
    agg_dict = {'min_coord': 'min', 'max_coord': 'max'}
    t_df = df.copy(deep=True)
    t_df['min_coord'] = t_df[['Start', 'End']].min(axis=1)
    t_df['max_coord'] = t_df[['Start', 'End']].max(axis=1)
    if how == 't':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id', 'transcript_id', 'transcript_name',
                   'tss_id', 'tes_id',
                   'new_transcript_id', 'original_transcript_id',
                   'original_transcript_name', 'ag1', 'ag2']
    elif how == 'g':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id']
    gb_cols = list(set(gb_cols)&(set(t_df.columns)))

    cols = gb_cols + ['min_coord', 'max_coord']
    t_df = t_df[cols]
    t_df = t_df.groupby(gb_cols, observed=True).agg(agg_dict).reset_index()
    t_df.rename({'min_coord': 'Start', 'max_coord': 'End'}, axis=1, inplace=True)
    if how == 't':
        t_df['Feature'] = 'transcript'
    elif how == 'g':
        t_df['Feature'] = 'gene'

    return t_df

In [4]:
df = pr.read_gtf('2_PY2_GM10493.gtf', as_df=True)

In [12]:
# noivel gene finder for esprsso
novel_gids = df.loc[(df.Source=='novel_isoform')&\
                    (df.gene_id=='NA')].gene_id.unique().tolist()
novel_gids = ['NA']

['NA']

In [58]:
l = df.loc[df.Source=='novel_isoform','gene_id'].unique().tolist()
print(len(l))
print(l[:5])

986
['ENSG00000273748.1', 'NA', 'ENSG00000115241.11', 'ENSG00000138002.16', 'ENSG00000171103.11']


In [52]:
# TODO try on one gene
# df = df.loc[df.gene_id=='ENSG00000296598.1']

print(len(df.index))
print(len(df.gene_id.unique()))

df = make_hier_entry(df, how='g')
print(len(df.loc[df.Feature=='gene'].index))

215498
12977
13025


In [53]:
df

,gene_id,Strand,Chromosome,Start,End,Feature
0,ENSG00000000003.16,-,chrX,100627107,100636806,gene
1,ENSG00000000419.14,-,chr20,50934869,50958532,gene
2,ENSG00000000457.14,-,chr1,169849630,169893959,gene
3,ENSG00000000460.17,+,chr1,169795042,169804347,gene
4,ENSG00000000938.13,-,chr1,27612063,27635185,gene
...,...,...,...,...,...,...
13020,NA,-,chr6,655943,167093593,gene
13021,NA,-,chr7,716619,149226170,gene
13022,NA,-,chr8,1482495,144012561,gene
13023,NA,-,chr9,23296,137032119,gene


In [55]:
df.loc[df.gene_id.duplicated(keep=False)].sort_values(by='gene_id')

,gene_id,Strand,Chromosome,Start,End,Feature
12976,NA,+,GL000219.1,98559,99612,gene
13002,NA,-,chr1,944205,244150826,gene
13003,NA,-,chr10,511506,129864305,gene
13004,NA,-,chr11,524290,125903200,gene
13005,NA,-,chr12,567393,131074524,gene
13006,NA,-,chr13,19674755,110728101,gene
13007,NA,-,chr14,20659870,106844796,gene
13008,NA,-,chr15,22166998,99896391,gene
13009,NA,-,chr16,46989,88983277,gene
13010,NA,-,chr17,400747,82212841,gene
